# Object Detection with TensorFlow Lite Model Maker

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [263]:
#!pip install cryptography --upgrade

In [264]:
#!pip install tflite-model-maker
#!pip install pycocotools

In [14]:
#pip install tflite_model_maker

Import the required packages.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os

from PIL import Image

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [4]:
!python --version
tf.__version__


Python 3.9.4


'2.8.0'

### Prepare the dataset

How to prepare your own CSV file and the minimum requirements for creating a valid dataset: [Preparing your training data](https://cloud.google.com/vision/automl/object-detection/docs/prepare).

Use a local path in your CSV file.

## Quickstart

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Pixel 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In [51]:
spec = model_spec.get('efficientdet_lite1')
spec.config.max_instances_per_image=200
spec.config.num_epochs=10       # start with small number for dev (1,5,or 10), optimal might be 50 or more epochs
spec.config.num_classes=2            # carotene (orange), white
spec.config.tflite_max_detections=100  # <<< default is 100
spec.config.label_map={1: "car", 2: "white"} 
#print(spec.config)

In [52]:
%cd ./data-set-maker/images-25-26-orange-white/train

/media/s0961819/neuro2019/UoE/phd-code/object-detection-api-tflite-yeast/data-set-maker/images-25-26-orange-white/train


In [53]:
%pwd

'/media/s0961819/neuro2019/UoE/phd-code/object-detection-api-tflite-yeast/data-set-maker/images-25-26-orange-white/train'

In [54]:
%ls *csv

train_labels-normed.csv*        train_labels_orange_white-rot.csv*
train_labels_orange_white.csv*


In [55]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('train_labels-normed.csv')

In [56]:
%cd ../../..

/media/s0961819/neuro2019/UoE/phd-code/object-detection-api-tflite-yeast


In [57]:
%pwd

'/media/s0961819/neuro2019/UoE/phd-code/object-detection-api-tflite-yeast'

In [58]:
print(type(train_data))
print(type(validation_data))
print(type(test_data))

<class 'tensorflow_examples.lite.model_maker.core.data_util.object_detector_dataloader.DataLoader'>
<class 'tensorflow_examples.lite.model_maker.core.data_util.object_detector_dataloader.DataLoader'>
<class 'tensorflow_examples.lite.model_maker.core.data_util.object_detector_dataloader.DataLoader'>


In [59]:
#
# this can take hours to run, so try with small data set and small number of epochs first, e.g. when debugging
#
# see spec.num_epochs above 

model = object_detector.create(train_data, model_spec=spec, batch_size=16, train_whole_model=True, validation_data=validation_data)

Epoch 1/10
11/11 [==============================] - 64s 4s/step - det_loss: 2.1187 - cls_loss: 1.1379 - box_loss: 0.0196 - reg_l2_loss: 0.0706 - loss: 2.1892 - learning_rate: 0.0139 - gradient_norm: 0.7854 - val_det_loss: 1.6831 - val_cls_loss: 1.1111 - val_box_loss: 0.0114 - val_reg_l2_loss: 0.0706 - val_loss: 1.7536
Epoch 2/10
11/11 [==============================] - 39s 4s/step - det_loss: 1.8811 - cls_loss: 1.0646 - box_loss: 0.0163 - reg_l2_loss: 0.0706 - loss: 1.9516 - learning_rate: 0.0186 - gradient_norm: 0.9216 - val_det_loss: 1.4535 - val_cls_loss: 1.0006 - val_box_loss: 0.0091 - val_reg_l2_loss: 0.0706 - val_loss: 1.5241
Epoch 3/10
11/11 [==============================] - 39s 4s/step - det_loss: 1.6080 - cls_loss: 0.9238 - box_loss: 0.0137 - reg_l2_loss: 0.0706 - loss: 1.6785 - learning_rate: 0.0164 - gradient_norm: 1.5265 - val_det_loss: 1.4129 - val_cls_loss: 0.9323 - val_box_loss: 0.0096 - val_reg_l2_loss: 0.0706 - val_loss: 1.4835
Epoch 4/10
11/11 [======================

**Evaluate the model with the test data.**

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [60]:
model.evaluate(test_data)  # AP average precision: higher is better

1/1 [==============================] - 4s 4s/step



{'AP': 0.013423249,
 'AP50': 0.04284271,
 'AP75': 0.0042027123,
 'APs': 0.0,
 'APm': 0.016950099,
 'APl': 0.00020120724,
 'ARmax1': 0.0076286765,
 'ARmax10': 0.033590216,
 'ARmax100': 0.059226528,
 'ARs': 0.0,
 'ARm': 0.05954325,
 'ARl': 0.1,
 'AP_/car': 0.021372067,
 'AP_/white': 0.0054744314}

In [37]:
pred=model.model.make_predict_function()

In [38]:
# Export TensorFlow (not Lite) model, see folders
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])

**Step 5.  Export as a TensorFlow Lite model.**

Default post-training quantization technique is full integer quantization.

In [39]:
%pwd

'/media/s0961819/neuro2019/UoE/phd-code/object-detection-api-tflite-yeast'

In [61]:
# Saves as Lite model, this might take a few minutes, will see model.tflite in current folder
model.export(export_dir='.')

**Step 6.  Evaluate the TensorFlow Lite model.**

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [26]:
model.evaluate_tflite('model.tflite', test_data)   # *** default filename ***

28/28 [==============================] - 86s 3s/step



{'AP': 0.01334385,
 'AP50': 0.0463233,
 'AP75': 0.0030537874,
 'APs': -1.0,
 'APm': 0.023046548,
 'APl': -1.0,
 'ARmax1': 0.005,
 'ARmax10': 0.04796875,
 'ARmax100': 0.07671875,
 'ARs': -1.0,
 'ARm': 0.07671875,
 'ARl': -1.0,
 'AP_/car': 0.012081001,
 'AP_/white': 0.014606699}